# 本ノートブックの目的

Lorenz63 modelのリアプノフ指数を計算すること．

# Import

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
from logging import INFO, StreamHandler, getLogger

logger = getLogger()
if not logger.hasHandlers():
    logger.addHandler(StreamHandler(sys.stdout))
logger.setLevel(INFO)

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import torch
from numpy.polynomial import polynomial as P
from src.lorenz63_model.lorenz63_model import Lorenz63
from src.lorenz63_model.utils.lorenz63_config import Lorenz63Config
from src.utils.random_seed_helper import set_seeds
from tqdm.notebook import tqdm

plt.rcParams["font.family"] = "serif"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = r":4096:8"  # to make calculations deterministic

# Define constant

In [ ]:
ROOT_DIR = pathlib.Path(os.environ["PYTHONPATH"]).parent.resolve()

fig_dir = f"{ROOT_DIR}/docs/data_assimilation/fig"
os.makedirs(fig_dir, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu").type

# Define method

In [ ]:
def calc_two_trajectory_distance_time_series(
    result1: torch.Tensor,
    result2: torch.Tensor,
) -> np.ndarray:

    assert result1.ndim == result2.ndim == 2

    num_tsteps = result1.shape[0]

    dis_time_series = []
    for it in range(num_tsteps):
        dis = torch.linalg.norm(result1[it] - result2[it])

        dis_time_series.append(dis.item())

    return np.array(dis_time_series)

# Run simulation

In [ ]:
set_seeds(seed=42, use_deterministic=True)

In [ ]:
cfg = Lorenz63Config(
    n_batch=2,
    noise_amplitude=0.00001,
    device=DEVICE,
    precision="double"
)

In [ ]:
model = Lorenz63(cfg, show_input_cfg_info=False)

In [ ]:
X0 = torch.tensor([11.2, 10.2, 33.2], dtype=model.real_dtype).to(model.device)

model.initialize(X=X0)

Xs, ts = [model.get_state()], [model.t]

dt = 0.001
output_dt = 0.01
end_time = 30

output_tsteps = torch.arange(output_dt, end_time + output_dt, output_dt)

for _ in tqdm(output_tsteps):
    model.integrate_n_steps(dt_per_step=dt, n_steps=int(output_dt / dt))
    Xs.append(model.get_state())
    ts.append(model.t)

ts = np.array(ts)

# Stack arrays along time dim
Xs = torch.stack(Xs, dim=1).squeeze()

# shape = (batch, time, (x, y, z))
logger.info(f"Shape of the result: {Xs.shape}")

# Plot simulation result

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 10))
for (i, ax), ylabel in zip(enumerate(axes), ["x", "y", "z"]):
    ax.plot(ts, Xs[0, :, i], label="Result 1")
    ax.plot(ts, Xs[1, :, i], label="Result 2")
    ax.set_xlabel("t")
    ax.set_ylabel(ylabel)

    ax.legend(loc=3)

plt.show()

# Calculate Lyapunov exponent

In [ ]:
dis_time_series = calc_two_trajectory_distance_time_series(Xs[0], Xs[1])

ts_end = 1700

linear_fit_coef = P.polyfit(ts[:ts_end], np.log(dis_time_series[:ts_end]), 1)
lyapunov_exponent = linear_fit_coef[1]

logger.info(f"Lyapunov exponent is {lyapunov_exponent}")

In [ ]:
linear_values = P.polyval(ts[:ts_end], linear_fit_coef)

plt.rcParams["font.size"] = 18

fig = plt.figure(figsize=(10, 6))

plt.plot(ts, np.log(dis_time_series), label=r"$y =$ ln ($\mathit{L}$)")
plt.plot(ts[:ts_end], linear_values, label=rf"$y = \lambda t + b$ ($\lambda$ = {lyapunov_exponent:.3f})")

plt.xlabel(r"$t$ : Time")
plt.ylabel(r"$y$")

plt.title(r"$\mathit{L}$ means Distance of two trajectorys at $t$")

plt.legend()

# plt.savefig(f"{fig_dir}/lyapunov_exponent_plot.png")

plt.show()